In [7]:
import pandas as pd
from sklearn import preprocessing

In [8]:
# Import raw data
data = pd.read_csv("../de_id_data_v2_6.6.2024.csv")
data

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/3435602546.py:2: DtypeWarning: Columns (3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../de_id_data_v2_6.6.2024.csv")


,appointment_id,age_deid,bmi_deid,ethnicity,ethnic_category_black,ethnic_category_unknown,gender_male,imd19_quintile,main_spoken_language_other,current_smoker,...,service_mapping_priority,service_mapping_surgical,weekswaiting,no_show,did_not_attend_risk,did_not_attend_risk_group,prediction_type,case_control,use_case,appt_date
0,10037229,12,27.45,NaN,0,0,1,4,0,1,...,0,0,NaN,1.0,0.172123,Moderate Risk,NaN,0.0,3,2023-02-09
1,10079786,52,27.45,NaN,0,1,1,2,0,0,...,0,0,NaN,0.0,0.264963,High Risk,First prediction,1.0,3,2024-03-20
2,10079786,52,39.45,NaN,1,0,0,2,0,0,...,0,0,NaN,0.0,0.197289,High Risk,Last prediction,1.0,3,2024-03-20
3,10098651,7,27.45,NaN,1,0,0,2,0,0,...,0,0,NaN,0.0,0.298824,High Risk,NaN,0.0,3,2023-03-16
4,10112206,22,18.00,NaN,0,0,0,2,1,1,...,0,0,NaN,0.0,0.244291,High Risk,NaN,0.0,3,2023-01-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473151,13516860,57,39.45,NaN,0,0,0,3,0,0,...,0,0,NaN,0.0,0.252709,High Risk,NaN,0.0,3,2022-12-19
473152,13525823,62,39.45,NaN,0,0,0,2,0,0,...,0,0,NaN,0.0,0.132705,Moderate Risk,NaN,0.0,3,2023-04-17
473153,13526514,47,39.45,NaN,1,0,0,1,0,1,...,0,1,NaN,0.0,0.196135,High Risk,NaN,0.0,3,2023-02-14
473154,13530398,52,39.45,NaN,0,0,1,2,0,0,...,0,1,NaN,0.0,0.249059,High Risk,NaN,0.0,3,2023-01-17


### Split out data for use case 1 (future appts)

In [9]:
# Split out data for use case 1
future_data = data.loc[data["use_case"]  == 1]
future_data

,appointment_id,age_deid,bmi_deid,ethnicity,ethnic_category_black,ethnic_category_unknown,gender_male,imd19_quintile,main_spoken_language_other,current_smoker,...,service_mapping_priority,service_mapping_surgical,weekswaiting,no_show,did_not_attend_risk,did_not_attend_risk_group,prediction_type,case_control,use_case,appt_date
7511,14943511,32,27.45,Asian,0,0,1,2,0,0,...,0,0,58.0,NaN,0.287193,High Risk,NaN,NaN,1,2024-05-08
22431,15742730,42,27.45,Not Recorded/Stated,0,1,0,1,0,0,...,0,0,5.0,NaN,0.351414,High Risk,NaN,NaN,1,2024-08-02
117173,16778162,22,21.70,White,0,0,0,2,0,0,...,0,1,67.0,NaN,0.575675,Very High Risk,NaN,NaN,1,2024-05-20
145632,16994098,32,39.45,Not Recorded/Stated,0,1,0,2,0,0,...,0,0,63.0,NaN,0.150000,Moderate Risk,NaN,NaN,1,2024-07-31
152249,17047096,62,27.45,Not Recorded/Stated,0,0,1,1,0,0,...,0,0,62.0,NaN,0.340443,High Risk,NaN,NaN,1,2024-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472875,20254158,32,21.70,White,0,0,1,3,0,0,...,0,0,4.0,NaN,0.365332,High Risk,NaN,NaN,1,2024-09-04
472876,20254160,82,27.45,Black,1,0,0,2,0,0,...,0,0,4.0,NaN,0.088879,Low Risk,NaN,NaN,1,2024-09-04
472877,20254163,82,27.45,Black,1,0,0,1,0,1,...,0,0,6.0,NaN,0.102636,Low Risk,NaN,NaN,1,2024-09-04
472880,20283951,32,27.45,Not Recorded/Stated,0,1,1,2,0,0,...,0,0,3.0,NaN,0.111294,Moderate Risk,NaN,NaN,1,2024-07-05


In [10]:
# Export data before transformation
future_data.to_csv("../future_appts_raw.csv", index=False)

In [11]:
# Count number of unique appointment IDs
future_data["appointment_id"].nunique()

15841

In [12]:
# Remove appointments missing appt_date
future_data = future_data.dropna(subset=["appt_date"])
future_data["appointment_id"].nunique()

15815

### Transform date from object to datetime

In [13]:
future_data["appt_date"] = pd.to_datetime(future_data["appt_date"])
future_data["appt_date"]

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/379191052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["appt_date"] = pd.to_datetime(future_data["appt_date"])


7511     2024-05-08
22431    2024-08-02
117173   2024-05-20
145632   2024-07-31
152249   2024-05-01
            ...    
472875   2024-09-04
472876   2024-09-04
472877   2024-09-04
472880   2024-07-05
472881   2024-10-21
Name: appt_date, Length: 15815, dtype: datetime64[ns]

### Transform categorical data to numeric data

In [14]:
future_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15815 entries, 7511 to 472881
Data columns (total 58 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   appointment_id                        15815 non-null  int64         
 1   age_deid                              15815 non-null  int64         
 2   bmi_deid                              15815 non-null  float64       
 3   ethnicity                             15815 non-null  object        
 4   ethnic_category_black                 15815 non-null  int64         
 5   ethnic_category_unknown               15815 non-null  int64         
 6   gender_male                           15815 non-null  int64         
 7   imd19_quintile                        15815 non-null  int64         
 8   main_spoken_language_other            15815 non-null  int64         
 9   current_smoker                        15815 non-null  int64         
 10 

In [15]:
# Encode the ethnicity column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(future_data["ethnicity"])

# Add the encoded column to the data
future_data["ethnicity_trans"] = label

future_data

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/2007977362.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["ethnicity_trans"] = label


,appointment_id,age_deid,bmi_deid,ethnicity,ethnic_category_black,ethnic_category_unknown,gender_male,imd19_quintile,main_spoken_language_other,current_smoker,...,service_mapping_surgical,weekswaiting,no_show,did_not_attend_risk,did_not_attend_risk_group,prediction_type,case_control,use_case,appt_date,ethnicity_trans
7511,14943511,32,27.45,Asian,0,0,1,2,0,0,...,0,58.0,NaN,0.287193,High Risk,NaN,NaN,1,2024-05-08,0
22431,15742730,42,27.45,Not Recorded/Stated,0,1,0,1,0,0,...,0,5.0,NaN,0.351414,High Risk,NaN,NaN,1,2024-08-02,3
117173,16778162,22,21.70,White,0,0,0,2,0,0,...,1,67.0,NaN,0.575675,Very High Risk,NaN,NaN,1,2024-05-20,5
145632,16994098,32,39.45,Not Recorded/Stated,0,1,0,2,0,0,...,0,63.0,NaN,0.150000,Moderate Risk,NaN,NaN,1,2024-07-31,3
152249,17047096,62,27.45,Not Recorded/Stated,0,0,1,1,0,0,...,0,62.0,NaN,0.340443,High Risk,NaN,NaN,1,2024-05-01,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472875,20254158,32,21.70,White,0,0,1,3,0,0,...,0,4.0,NaN,0.365332,High Risk,NaN,NaN,1,2024-09-04,5
472876,20254160,82,27.45,Black,1,0,0,2,0,0,...,0,4.0,NaN,0.088879,Low Risk,NaN,NaN,1,2024-09-04,1
472877,20254163,82,27.45,Black,1,0,0,1,0,1,...,0,6.0,NaN,0.102636,Low Risk,NaN,NaN,1,2024-09-04,1
472880,20283951,32,27.45,Not Recorded/Stated,0,1,1,2,0,0,...,0,3.0,NaN,0.111294,Moderate Risk,NaN,NaN,1,2024-07-05,3


In [16]:
# Encode the tfc_name column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(future_data["tfc_name"])

# Add the encoded column to the data
future_data["tfc_name_trans"] = label

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/2065304299.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["tfc_name_trans"] = label


In [17]:
# Encode the patient_on_multiple_pathways column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(future_data["patient_on_multiple_pathways"])

# Add the encoded column to the data
future_data["patient_on_multiple_pathways_trans"] = label

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/156116874.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["patient_on_multiple_pathways_trans"] = label


In [18]:
# Encode the appt_month column

month_mapping ={
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

# Map the month names to numbers
future_data["appt_month_trans"] = future_data["appt_month"].map(month_mapping)

future_data[['appt_month', 'appt_month_trans']]


/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/4180195568.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["appt_month_trans"] = future_data["appt_month"].map(month_mapping)


,appt_month,appt_month_trans
7511,October,10
22431,August,8
117173,May,5
145632,June,6
152249,May,5
...,...,...
472875,September,9
472876,September,9
472877,September,9
472880,July,7


In [19]:
# Encode the hospital_service_display column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(future_data["hospital_service_display"])

# Add the encoded column to the data
future_data["hospital_service_display_trans"] = label

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/3746783091.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["hospital_service_display_trans"] = label


In [20]:
# Encode the service_mapping column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(future_data["service_mapping"])

# Add the encoded column to the data
future_data["service_mapping_trans"] = label

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/2248520237.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["service_mapping_trans"] = label


In [21]:
# Encode the reason_display column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(future_data["reason_display"])

# Add the encoded column to the data
future_data["reason_display_trans"] = label

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/409614947.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["reason_display_trans"] = label


In [22]:
# Find out the codes for reason_display that contain 'adult ENT'
print(future_data[future_data["reason_display"].str.contains("adult ENT", case=False, na=False)]["reason_display"].unique())
ent_appts = future_data[future_data["reason_display"].str.contains("adult ENT", case=False, na=False)]
ent_codes = ent_appts["reason_display_trans"].unique()
ent_codes

['Adult ENT F/Up' 'Adult ENT New' 'Adult ENT Thyroid F/Up'
 'Adult ENT Thyroid New' 'Adult ENT Balance New' 'Adult ENT Results F/Up'
 'Adult ENT Post Op F/Up' 'Adult ENT Voice Clinic New'
 'Adult ENT Voice Clinic F/Up' 'Adult ENT Pre-Assessment F/Up'
 'Adult ENT Nurse Skin Test New' 'Adult ENT Voice Osteopath F/Up'
 'Adult ENT Voice Osteopath New']


array([ 1,  2,  7,  8,  0,  6,  4, 10,  9,  5,  3, 11, 12])

In [23]:
# Encode the appointment_type column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(future_data["appointment_type"])

# Add the encoded column to the data
future_data["appointment_type_trans"] = label

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/1623803260.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["appointment_type_trans"] = label


In [24]:
# Encode the did_not_attend_risk_group column

risk_mapping ={
    "Low Risk": 1,
    "Moderate Risk": 2,
    "High Risk": 3,
    "Very High Risk": 4,
}

# Map the month names to numbers
future_data["did_not_attend_risk_group_trans"] = future_data["did_not_attend_risk_group"].map(risk_mapping)

future_data[['did_not_attend_risk_group', 'did_not_attend_risk_group_trans']]

/var/folders/0t/dc47jt692_qc1qb7bqkgylhw0000gn/T/ipykernel_82397/2086460805.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data["did_not_attend_risk_group_trans"] = future_data["did_not_attend_risk_group"].map(risk_mapping)


,did_not_attend_risk_group,did_not_attend_risk_group_trans
7511,High Risk,3
22431,High Risk,3
117173,Very High Risk,4
145632,Moderate Risk,2
152249,High Risk,3
...,...,...
472875,High Risk,3
472876,Low Risk,1
472877,Low Risk,1
472880,Moderate Risk,2


In [25]:
future_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15815 entries, 7511 to 472881
Data columns (total 67 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   appointment_id                        15815 non-null  int64         
 1   age_deid                              15815 non-null  int64         
 2   bmi_deid                              15815 non-null  float64       
 3   ethnicity                             15815 non-null  object        
 4   ethnic_category_black                 15815 non-null  int64         
 5   ethnic_category_unknown               15815 non-null  int64         
 6   gender_male                           15815 non-null  int64         
 7   imd19_quintile                        15815 non-null  int64         
 8   main_spoken_language_other            15815 non-null  int64         
 9   current_smoker                        15815 non-null  int64         
 10 

### Export the transformed use case 1 data

In [26]:
# Dropped all transformed columns
future_data_trans = future_data.drop(["ethnicity",
"tfc_name",
"patient_on_multiple_pathways",
"appt_month",
"hospital_service_display",
"service_mapping",
#"reason_display",
#"appointment_type",
"did_not_attend_risk_group"], axis =1)

# Export data for use case 1 to CSV
future_data_trans.to_csv("../future_appts_trans.csv", index=False)

In [27]:
# Filter for adult ENT appointments only
ent_codes = [1,  2,  7,  8,  0,  6,  4, 10,  9,  5,  3, 11, 12]
future_data_trans_ent = future_data_trans[future_data_trans["reason_display_trans"].isin(ent_codes)]

future_data_trans_ent.to_csv("../future_appts_trans_ENT.csv", index=False)